In [1]:
from generation.GeneratedTestSaver import GeneratedTestSaver
from history_scanner.GitHistoryDataSetParser import GitHistoryDataSetParser
from generation.Model import Model

In [2]:
# Load data
parser = GitHistoryDataSetParser("dataset_repos/pydriller", branch="master")
data = parser.load_data("dataset_repos/data/pydriller_commits.dat")
model = Model()

In [3]:
# Parse data
parser = GitHistoryDataSetParser("dataset_repos/pydriller", branch="master")
data = parser.parse_data()

created git_repository and other important classes: 
['CommitVisitor.py', 'git_repository.py']
['git_repository_tests.py']

git repo now has get_commit: 
['git_repository.py']
['git_repository_tests.py']

add modification, implemented files(): 
['modification.py', 'git_repository.py']
['git_repository_tests.py']

first visitor works!: 
['git_repository.py']
['git_repository_tests.py']

improvements on git_repository: 
['modification.py']
['git_repository_tests.py']

added filters. new tests too: 
['repository_mining.py']
['git_repository_tests.py', 'repository_mining_tests.py']

now it handles different branches: 
['git_repository.py']
['git_repository_tests.py', 'repository_mining_tests.py']

moving from unittest to pytest: 
['repository_mining.py']
['test_git_repository.py', 'test_repository_mining.py']

now it creates a new branch when checking out. More tests too: 
['commit.py', 'repository_mining.py', 'git_repository.py']
['test_between_dates.py', 'test_dates_and_timezones.py', 't


KeyboardInterrupt



In [ ]:
print(str(data[-5]))

In [ ]:
print("\n".join(f.source for f in data[-5].test_files))

In [ ]:
prompts = [prompt for prompt in data[-5].construct_prompt()]

In [ ]:
generation = model.generate_test(str(prompts[0]))

In [ ]:
def print_generation(generation):
    for gen in generation:
        print(gen)

In [ ]:
import ast
parse = ast.parse(data[-5].source_files[0].source)

In [ ]:
def search_in_body(parse):
    for expr in parse.body:
        if isinstance(expr, ast.ClassDef):
            print(expr.name)
            search_in_body(expr)
        if isinstance(expr, ast.FunctionDef):
            print(expr.name)

In [ ]:
search_in_body(parse)

# Fixing imports

In [ ]:
import subprocess

In [ ]:
out = subprocess.run("pytest C:\\Users\\Ivan\\PycharmProjects\\auto-test-generator\\dataset_repos\\pydriller\\generated_tests\\test_2929a6aa-63c3-423f-9cf9-bb20187aa9fd.py", shell=True, stdout=subprocess.PIPE,cwd="C:\\Users\\Ivan\\PycharmProjects\\auto-test-generator\\dataset_repos\\pydriller").stdout.decode()

In [ ]:
print(out)

In [ ]:
import re
matches  = re.findall(r"NameError:\sname '(\w+)' is not defined", out)
print(matches)

In [ ]:
pass_out = out = subprocess.run("pytest C:\\Users\\Ivan\\PycharmProjects\\auto-test-generator\\dataset_repos\\pydriller\\generated_tests\\test_2929a6aa-63c3-423f-9cf9-bb20187aa9fd.py", shell=True, stdout=subprocess.PIPE,cwd="C:\\Users\\Ivan\\PycharmProjects\\auto-test-generator\\dataset_repos\\pydriller").stdout.decode()


In [ ]:
def extract_group(tup):
        return [i for i in tup if i is not None]

In [ ]:
#print(out)
errors = re.search(r"((?P<failedNumber>\d)+ failed)|((?P<passedNumber>\d+) passed)|((?P<warningsNumber>\d+) warning)|((?P<errorNumber>\d+) error)/g", out)
reg = re.compile(r'=.+((?P<failedNumber>\d+) warning)')
print(reg.search(out).groups('failedNumber')[1])

## Crawling for import

In [ ]:
import os
base_dir = "C:\\Users\\Ivan\\PycharmProjects\\auto-test-generator\\dataset_repos\\pydriller"

In [ ]:
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(base_dir):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames if file.endswith(".py")]
print(listOfFiles)

In [ ]:
def find_import(files, searched_entries):
    for file in files:
        with open(file, 'r') as f:
            head = f.readlines()
        for line in head:
            found_strings = []
            for s in searched_entries:
                if f'import' in line and s in line:
                    print(line)
                    found_strings.append(s)
            [searched_entries.remove(s) for s in found_strings]
        if len(searched_entries) == 0:
            break

In [ ]:
find_import(listOfFiles, set(matches))

# Model improvements

In [5]:
prompt = '''
# Copyright 2018 Davide Spadini
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
This module contains all the classes regarding a specific commit, such as
Commit, Modification,
ModificationType and Method.
"""

import logging
from _datetime import datetime
from enum import Enum
from pathlib import Path
from typing import List, Set, Dict

import lizard
from git import Repo, Diff, Git, Commit as GitCommit

from pydriller.domain.developer import Developer

logger = logging.getLogger(__name__)

NULL_TREE = '4b825dc642cb6eb9a060e54bf8d69288fbee4904'


class ModificationType(Enum):
    """
    Type of Modification. Can be ADD, COPY, RENAME, DELETE, MODIFY or UNKNOWN.
    """

    ADD = 1
    COPY = 2
    RENAME = 3
    DELETE = 4
    MODIFY = 5
    UNKNOWN = 6


class Method:  # pylint: disable=R0902
    """
    This class represents a method in a class. Contains various information
    extracted through Lizard.
    """

    def __init__(self, func):
        """
        Initialize a method object. This is calculated using Lizard: it parses
        the source code of all the modifications in a commit, extracting
        information of the methods contained in the file (if the file is a
        source code written in one of the supported programming languages).
        """

        self.name = func.name
        self.long_name = func.long_name
        self.filename = func.filename
        self.nloc = func.nloc
        self.complexity = func.cyclomatic_complexity
        self.token_count = func.token_count
        self.parameters = func.parameters
        self.start_line = func.start_line
        self.end_line = func.end_line
        self.fan_in = func.fan_in
        self.fan_out = func.fan_out
        self.general_fan_out = func.general_fan_out
        self.length = func.length
        self.top_nesting_level = func.top_nesting_level


class Modification:  # pylint: disable=R0902
    """
    This class contains information regarding a modified file in a commit.
    """

    def __init__(self, old_path: str, new_path: str,
                 change_type: ModificationType,
                 diff_and_sc: Dict[str, str]):
        """
        Initialize a modification. A modification carries on information
        regarding the changed file. Normally, you shouldn't initialize a new
        one.
        """
        self._old_path = Path(old_path) if old_path is not None else None
        self._new_path = Path(new_path) if new_path is not None else None
        self.change_type = change_type
        self.diff = diff_and_sc['diff']
        self.source_code = diff_and_sc['source_code']
        self.source_code_before = diff_and_sc['source_code_before']

        self._nloc = None
        self._complexity = None
        self._token_count = None
        self._function_list = []

    @property
    def added(self) -> int:
        """
        Return the total number of added lines in the file.

        :return: int lines_added
        """
        added = 0
        for line in self.diff.replace('\r', '').split("\n"):
            if line.startswith('+') and not line.startswith('+++'):
                added += 1
        return added

    @property
    def removed(self):
        """
        Return the total number of deleted lines in the file.

        :return: int lines_deleted
        """
        removed = 0
        for line in self.diff.replace('\r', '').split("\n"):
            if line.startswith('-') and not line.startswith('---'):
                removed += 1
        return removed

    @property
    def old_path(self):
        """
        Old path of the file. Can be None if the file is added.

        :return: str old_path
        """
        if self._old_path:
            return str(self._old_path)
        return self._old_path

    @property
    def new_path(self):
        """
        New path of the file. Can be None if the file is deleted.

        :return: str new_path
        """
        if self._new_path:
            return str(self._new_path)
        return self._new_path

    @property
    def filename(self) -> str:
        """
        Return the filename. Given a path-like-string (e.g.
        "/Users/dspadini/pydriller/myfile.py") returns only the filename
        (e.g. "myfile.py")

        :return: str filename
        """
        if self._new_path is not None and str(self._new_path) != "/dev/null":
            path = self._new_path
        else:
            path = self._old_path

        return path.name

    @property
    def nloc(self) -> int:
        """
        Calculate the LOC of the file.

        :return: LOC of the file
        """
        self._calculate_metrics()
        return self._nloc

    @property
    def complexity(self) -> int:
        """
        Calculate the Cyclomatic Complexity of the file.

        :return: Cyclomatic Complexity of the file
        """
        self._calculate_metrics()
        return self._complexity

    @property
    def token_count(self) -> int:
        """
        Calculate the token count of functions.

        :return: token count
        """
        self._calculate_metrics()
        return self._token_count

    @property
    def methods(self) -> List[Method]:
        """
        Return the list of methods in the file. Every method
        contains various information like complexity, loc, name,
        number of parameters, etc.

        :return: list of methods
        """
        self._calculate_metrics()
        return self._function_list

    def _calculate_metrics(self):
        if self.source_code and self._nloc is None:
            l = lizard.analyze_file.analyze_source_code(self.filename,
                                                        self.source_code)

            self._nloc = l.nloc
            self._complexity = l.CCN
            self._token_count = l.token_count

            for func in l.function_list:
                self._function_list.append(Method(func))

    def __eq__(self, other):
        if not isinstance(other, Modification):
            return NotImplemented
        if self is other:
            return True
        return self.__dict__ == other.__dict__

    def __str__(self): # pragma: no cover
        return (
            'MODIFICATION\n' +
            'Old Path: {}\n'.format(self.old_path) +
            'New Path: {}\n'.format(self.new_path) +
            'Type: {}\n'.format(self.change_type.name) +
            'Diff: {}\n'.format(self.diff) +
            'Source code before: {}\n'.format(self.source_code_before) +
            'Source code: {}\n'.format(self.source_code)
        )


class Commit:
    """
    Class representing a Commit. Contains all the important information such
    as hash, author, dates, and modified files.
    """

    def __init__(self, commit: GitCommit, project_path: Path,
                 main_branch: str) -> None:
        """
        Create a commit object.

        :param commit: GitPython Commit object
        :param project_path: path to the project (temporary folder in case
            of a remote repository)
        :param main_branch: main branch of the repo
        """
        self._c_object = commit
        self._main_branch = main_branch
        self.project_path = project_path

        self._modifications = None
        self._branches = None

    @property
    def hash(self) -> str:
        """
        Return the SHA of the commit.

        :return: str hash
        """
        return self._c_object.hexsha

    @property
    def author(self) -> Developer:
        """
        Return the author of the commit as a Developer object.

        :return: author
        """
        return Developer(self._c_object.author.name,
                         self._c_object.author.email)

    @property
    def committer(self) -> Developer:
        """
        Return the committer of the commit as a Developer object.

        :return: committer
        """
        return Developer(self._c_object.committer.name,
                         self._c_object.committer.email)

    @property
    def project_name(self) -> str:
        """
        Return the project name.

        :return: project name
        """
        return self.project_path.name

    @property
    def author_date(self) -> datetime:
        """
        Return the authored datetime.

        :return: datetime author_datetime
        """
        return self._c_object.authored_datetime

    @property
    def committer_date(self) -> datetime:
        """
        Return the committed datetime.

        :return: datetime committer_datetime
        """
        return self._c_object.committed_datetime

    @property
    def author_timezone(self) -> int:
        """
        Author timezone expressed in seconds from epoch.

        :return: int timezone
        """
        return self._c_object.author_tz_offset

    @property
    def committer_timezone(self) -> int:
        """
        Author timezone expressed in seconds from epoch.

        :return: int timezone
        """
        return self._c_object.committer_tz_offset

    @property
    def msg(self) -> str:
        """
        Return commit message.

        :return: str commit_message
        """
        return self._c_object.message.strip()

    @property
    def parents(self) -> List[str]:
        """
        Return the list of parents SHAs.

        :return: List[str] parents
        """
        parents = []
        for p in self._c_object.parents:
            parents.append(p.hexsha)
        return parents

    @property
    def merge(self) -> bool:
        """
        Return True if the commit is a merge, False otherwise.

        :return: bool merge
        """
        return len(self._c_object.parents) > 1

    @property
    def modifications(self) -> List[Modification]:
        """
        Return a list of modified files.

        :return: List[Modification] modifications
        """
        if self._modifications is None:
            self._modifications = self._get_modifications()

        return self._modifications

    def _get_modifications(self):
        repo = Repo(str(self.project_path))
        commit = self._c_object

        if len(self.parents) == 1:
            # the commit has a parent
            diff_index = self._c_object.parents[0].diff(commit,
                                                        create_patch=True)
        elif len(self.parents) > 1:
            # if it's a merge commit, the modified files of the commit are the
            # conflicts. This because if the file is not in conflict,
            # pydriller will visit the modification in one of the previous
            # commits. However, parsing the output of a combined diff (that
            # returns the list of conflicts) is challenging: so, right now,
            # I will return an empty array, in the meanwhile I will try to
            # find a way to parse the output.
            # c_git = Git(str(self.project_path))
            # d = c_git.diff_tree("--cc", commit.hexsha, '-r', '--abbrev=40',
            #                     '--full-index', '-M', '-p', '--no-color')
            diff_index = []
        else:
            # this is the first commit of the repo. Comparing it with git
            # NULL TREE
            parent = repo.tree(NULL_TREE)
            diff_index = parent.diff(commit.tree, create_patch=True)

        return self._parse_diff(diff_index)

    def _parse_diff(self, diff_index) -> List[Modification]:
        modifications_list = []
        for diff in diff_index:
            old_path = diff.a_path
            new_path = diff.b_path
            change_type = self._from_change_to_modification_type(diff)

            diff_and_sc = {
                'diff': self._get_decoded_str(diff.diff),
                'source_code_before': self._get_decoded_sc_str(
                    diff.a_blob),
                'source_code': self._get_decoded_sc_str(
                    diff.b_blob)
            }

            modifications_list.append(Modification(old_path, new_path,
                                                   change_type, diff_and_sc))

        return modifications_list

    def _get_decoded_str(self, diff):
        try:
            return diff.decode('utf-8', 'ignore')
        except (UnicodeDecodeError, AttributeError, ValueError):
            logger.debug('Could not load the diff of a '
                         'file in commit %s', self._c_object.hexsha)
            return None

    def _get_decoded_sc_str(self, diff):
        try:
            return diff.data_stream.read().decode('utf-8', 'ignore')
        except (UnicodeDecodeError, AttributeError, ValueError):
            logger.debug('Could not load source code of a '
                         'file in commit %s', self._c_object.hexsha)
            return None

    @property
    def in_main_branch(self) -> bool:
        """
        Return True if the commit is in the main branch, False otherwise.

        :return: bool in_main_branch
        """
        return self._main_branch in self.branches

    @property
    def branches(self) -> Set[str]:
        """
        Return the set of branches that contain the commit.

        :return: set(str) branches
        """
        if self._branches is None:
            self._branches = self._get_branches()

        return self._branches

    def _get_branches(self):
        c_git = Git(str(self.project_path))
        branches = set()
        for branch in set(c_git.branch('--contains', self.hash).split('\n')):
            branches.add(branch.strip().replace('* ', ''))
        return branches

    # pylint disable=R0902
    def _from_change_to_modification_type(self, diff: Diff):
        if diff.new_file:
            return ModificationType.ADD
        if diff.deleted_file:
            return ModificationType.DELETE
        if diff.renamed_file:
            return ModificationType.RENAME
        if diff.a_blob and diff.b_blob and diff.a_blob != diff.b_blob:
            return ModificationType.MODIFY

        return ModificationType.UNKNOWN

    def __eq__(self, other):
        if not isinstance(other, Commit):
            return NotImplemented
        if self is other:
            return True

        return self.__dict__ == other.__dict__

    def __str__(self): # pragma: no cover
        return (
            'Hash: {}\n'.format(self.hash) +
            'Author: {}\n'.format(self.author.name) +
            'Author email: {}\n'.format(self.author.email) +
            'Committer: {}\n'.format(self.committer.name) +
            'Committer email: {}\n'.format(self.committer.email) +
            'Author date: {}\n'.format(
                self.author_date.strftime("%Y-%m-%d %H:%M:%S")) +
            'Committer date: {}\n'.format(
                self.committer_date.strftime("%Y-%m-%d %H:%M:%S")) +
            'Message: {}\n'.format(self.msg) +
            'Parent: {}\n'.format("\n".join(map(str, self.parents))) +
            'Merge: {}\n'.format(self.merge) +
            'Modifications: \n{}'.format(
                "\n".join(map(str, self.modifications))) +
            'Branches: \n{}'.format("\n".join(map(str, self.branches))) +
            'In main branch: {}\n'.format(self.in_main_branch)
        )
"""
Test which tests class Commit and dmm_unit_complexity. For the test use test-repos/dmm-test-repo. Assert the dmm_unit_complexity to be 1.0
"""
def test_
'''

In [6]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
response = openai.Completion.create(
                engine="code-davinci-002",
                prompt=prompt,
                temperature=0,
                max_tokens=256,
                n=4,
                frequency_penalty=0,
                presence_penalty=0,
                echo=True,
                stop=["#"]
)
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n# Copyright 2018 Davide Spadini\n#\n# Licensed under the Apache License, Version 2.0 (the \"License\");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#    http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an \"AS IS\" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n\n\"\"\"\nThis module contains all the classes regarding a specific commit, such as\nCommit, Modification,\nModificationType and Method.\n\"\"\"\n\nimport logging\nfrom _datetime import datetime\nfrom enum import Enum\nfrom pathlib import Path\nfrom typing import List, Set, Dict\n\nimport l

In [ ]:
import ast
gen_code = response['choices'][3]['text']

In [ ]:
from generation.Model import Model

m = Model()
code = m.generate_test(prompt)

In [ ]:
res = m.complete(prompt)

In [ ]:
import re
import ast
def compile_code(code, depth=90):
    if depth == 0:
        print("Couldn't make compilable for code")
        return code
    try:
        ast.parse(code)
        return code
    except SyntaxError as e:
        lines = code.split("\n")
        line_number = re.findall(r"line (\d+)", e.__str__())[0]
        print("Syntax Error:", line_number, e)
        lines.pop(int(line_number) - 1)
        return compile_code("\n".join(lines), depth - 1)
    except NameError as e:
        print("Name error: ", e)
        return code

In [ ]:
compile_code(res[0])

In [ ]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
response = openai.Edit.create(
            model="code-davinci-edit-001",
            input='''
###Python3
import pytest

def test_calculator_can_calculate_equation_with_plus_operator():

    operators = [PlusOperator()]

    calc = Calculator(operators)

    assert calc.calculate("1 + 2 + 4 + 5") == 12

    assert calc.calculate("10+10+10") == 30

"""

Added basic Calucator functionality.

Calculator should be able to calculate equations with PlusOperator as follows:

1+2+4+5=12

10+10+10=30

The tests are testing: class Calculator

"""

def test_calculator_can_calculate_equation_with_plus_operator():

    operators = [PlusOperator()]

    calc = Calculator(operators)

    assert calc.calculate("1+2+4+5") == 12

    assert calc.calculate("10+10+10
''',
            instruction="Fix compilation",
            temperature=0,
            top_p=1
        )

In [ ]:
print(response['choices'][0]['text'])

In [7]:
prompt='''


import pytest

def test_multiply_operation():

    operators = [MultiplyOperator()]

    calc = Calculator(operators)

    result = calc.calculate("2*4=")



    assert 8 == result

def test_plus_and_multiply_operation():

    operators = [PlusOperator(), MultiplyOperator()]

    calc = Calculator(operators)



    for line in ["2+10*3=", "2*10+3=", "5+6*7="]:

        print(line, end=' ')

        result = calc.calculate(line)



        assert int(line[:-1].replace('+', '').replace('*', '')) + (int(line[0]) * int(line[-2])) == result

# Generate asserts

import pytest

"""

Added support for MultiplyOperations.

Calculator should be able to handle equations of:

2*4=8

and support both multiplication and sumarization like:

2+10*2=22

The tests are testing: class Calculator

"""





def test_multiply_operation(): ...





def test_plus_and_multiply_operation(): ...
'''

In [9]:
openai.Edit.create(
                model="code-davinci-edit-001",
                input=prompt,
                instruction="Fix compilation",
                temperature=0,
                top_p=1
            )

<OpenAIObject edit at 0x1f4fe83be50> JSON: {
  "choices": [
    {
      "index": 0,
      "text": "import pytest\ndef test_multiply_operation():\n    operators = [MultiplyOperator()]\n    calc = Calculator(operators)\n    result = calc.calculate(\"2*4=\")\n\n    assert 8 == result\ndef test_plus_and_multiply_operation():\n    operators = [PlusOperator(), MultiplyOperator()]\n    calc = Calculator(operators)\n\n    for line in [\"2+10*3=\", \"2*10+3=\", \"5+6*7=\"]:\n        print(line, end=' ')\n        result = calc.calculate(line)\n\n        assert int(line[:-1].replace('+', '').replace('*', '')) + (int(line[0]) * int(line[-2])) == result\n# Generate asserts\nimport pytest\n\"\"\"\nAdded support for MultiplyOperations.\nCalculator should be able to handle equations of:\n2*4=8\nand support both multiplication and sumarization like:\n2+10*2=22\nThe tests are testing: class Calculator\n\"\"\"\n\n \ndef test_multiply_operation(): ...\n\n\n \n\ndef test_plus_and_multiply_operation(): ...\